In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
from ipynb.fs.full.features import features_independientes_precio, features_dependientes_precio, features_de_csvs, columna_a_ohe

df_train = pd.read_csv('./data/train.csv')
# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

# df_train, df_test = features_de_csvs(df_train, df_test)

# df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.15)

In [2]:
df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)

df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)

df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

df_test_f, cols_tipodepropiedad_ohe = columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)

df_train_f = columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)

llenar_nulls(df_train_f)
llenar_nulls(df_test_f)

## LightGBM model

In [3]:
from sklearn.model_selection import train_test_split

class LightGBMWrapper(lgb.LGBMRegressor):
    
    def fit(self, x, y):        
        x_train, x_test, y_train, y_test = train_test_split(x, y)
        return super(LightGBMWrapper, self).fit(x_train, y_train)
    
    def predict(self, X):
        return super(LightGBMWrapper, self).predict(X, 
               num_iteration=self.best_iteration_)

hps = {'bagging_fraction': 0.806451877022587,
 'bagging_freq': 62.0,
 'feature_fraction': 0.5379925983440028,
 'learning_rate': 0.1363027714646826,
 'max_depth': 11.0,
 'num_leaves': 113.0,
 'test_size': 0.09575190901892519}

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae', # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': int(hps['num_leaves']),
    'learning_rate': hps['learning_rate'],
    'feature_fraction': hps['feature_fraction'],
    'bagging_fraction': hps['bagging_fraction'],
    'bagging_freq': int(hps['bagging_freq']),
    'max_depth': int(hps['max_depth']),
    'verbose': 0
}

lgb_m = LightGBMWrapper(**params)

## Keras model

In [4]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation

def keras_modelo():    
    model = Sequential()
    model.add(BatchNormalization())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=1, activation='linear'))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'], validation_split=0.1)
    return model

keras_m = KerasRegressor(build_fn=keras_modelo, epochs=10)

Using TensorFlow backend.


## XGBoost

In [5]:
import xgboost as xgb

class XGBoostWrapper(xgb.XGBRegressor):
    
    def fit(self, x, y):
        return super(xgb.XGBRegressor, self).fit(x, y, early_stopping_rounds=2, eval_metric='mae', eval_set=[(x, y)])
    
    def predict(self, X):
        return super(xgb.XGBRegressor, self).predict(X)


hps = {'alpha': 20.91434940058063,
       'colsample_bytree': 0.65,
       'learning_rate': 0.14,
       'max_depth': int(16.0),
       'n_estimators': int(150.0),
       'test_size': 0.2,
       'early_stopping_rounds': 5,
       'n_jobs': 4}


n_estimators = int(hps['n_estimators'])
max_depth = int(hps['max_depth'])

xgb_m = XGBoostWrapper(**hps)

## Stacking

In [6]:
from sklearn.preprocessing import MinMaxScaler

from vecstack import StackingTransformer



features = ['habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 
#             'lat', 'lng',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'porcentaje_metros', 'diferencia_metros', 'promedio_precio_ciudad', 
                 'promedio_por_mes', 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'promedio_precio_hbg_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
                 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 
                 'promedio_precio_tipo_propiedad_ciudad_gen', 'promedio_precio_hbg_tipo_propiedad_provincia']

features += features_test

features += cols_tipodepropiedad_ohe + cols_provincia_ohe

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.001)

modelos = [('xgboost', xgb_m), 
#            ('keras', keras_m), 
           ('lightgbm', lgb_m)]

stack = StackingTransformer(modelos, regression=True, verbose=2, n_folds=6)

stack = stack.fit(x_train, y_train)

s_train = stack.transform(x_train)
s_test = stack.transform(x_test)

task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [2]

estimator  0: [xgboost: XGBoostWrapper]
[13:31:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:2.18892e+06
Will train until validation_0-mae hasn't improved in 2 rounds.
[1]	validation_0-mae:1.88947e+06
[2]	validation_0-mae:1.6331e+06
[3]	validation_0-mae:1.41342e+06
[4]	validation_0-mae:1.22562e+06
[5]	validation_0-mae:1.06527e+06
[6]	validation_0-mae:929240
[7]	validation_0-mae:814510
[8]	validation_0-mae:718409
[9]	validation_0-mae:637504
[10]	validation_0-mae:569539
[11]	validation_0-mae:514258
[12]	validation_0-mae:468803
[13]	validation_0-mae:429418
[14]	validation_0-mae:397346
[15]	validation_0-mae:371746
[16]	validation_0-mae:349020
[17]	validation_0-mae:330626
[18]	validation_0-mae:315000
[19]	validation_0-mae:300747
[20]	validation_0-mae:290184
[21]	validation_0-mae:279957
[22]	validati

[111]	validation_0-mae:117346
[112]	validation_0-mae:116706
[113]	validation_0-mae:116482
[114]	validation_0-mae:115801
[115]	validation_0-mae:115436
[116]	validation_0-mae:115187
[117]	validation_0-mae:114530
[118]	validation_0-mae:113464
[119]	validation_0-mae:112653
[120]	validation_0-mae:111980
[121]	validation_0-mae:111632
[122]	validation_0-mae:111299
[123]	validation_0-mae:110478
[124]	validation_0-mae:109949
[125]	validation_0-mae:109297
[126]	validation_0-mae:108532
[127]	validation_0-mae:108049
[128]	validation_0-mae:107373
[129]	validation_0-mae:106822
[130]	validation_0-mae:106302
[131]	validation_0-mae:106030
[132]	validation_0-mae:105155
[133]	validation_0-mae:104741
[134]	validation_0-mae:104293
[135]	validation_0-mae:103819
[136]	validation_0-mae:103218
[137]	validation_0-mae:102988
[138]	validation_0-mae:102606
[139]	validation_0-mae:101875
[140]	validation_0-mae:101193
[141]	validation_0-mae:100947
[142]	validation_0-mae:100439
[143]	validation_0-mae:99868.8
[144]	val

[74]	validation_0-mae:142029
[75]	validation_0-mae:141517
[76]	validation_0-mae:141298
[77]	validation_0-mae:140608
[78]	validation_0-mae:138671
[79]	validation_0-mae:137313
[80]	validation_0-mae:136573
[81]	validation_0-mae:135761
[82]	validation_0-mae:135570
[83]	validation_0-mae:134859
[84]	validation_0-mae:134122
[85]	validation_0-mae:132989
[86]	validation_0-mae:132445
[87]	validation_0-mae:132098
[88]	validation_0-mae:130908
[89]	validation_0-mae:130461
[90]	validation_0-mae:128805
[91]	validation_0-mae:127755
[92]	validation_0-mae:126895
[93]	validation_0-mae:126271
[94]	validation_0-mae:125141
[95]	validation_0-mae:123008
[96]	validation_0-mae:122076
[97]	validation_0-mae:121645
[98]	validation_0-mae:121157
[99]	validation_0-mae:119718
[100]	validation_0-mae:119005
[101]	validation_0-mae:118671
[102]	validation_0-mae:117866
[103]	validation_0-mae:117213
[104]	validation_0-mae:116450
[105]	validation_0-mae:115776
[106]	validation_0-mae:114005
[107]	validation_0-mae:113727
[108]	

[36]	validation_0-mae:201036
[37]	validation_0-mae:198483
[38]	validation_0-mae:195765
[39]	validation_0-mae:192778
[40]	validation_0-mae:190315
[41]	validation_0-mae:187517
[42]	validation_0-mae:185308
[43]	validation_0-mae:182969
[44]	validation_0-mae:181192
[45]	validation_0-mae:179888
[46]	validation_0-mae:178343
[47]	validation_0-mae:176959
[48]	validation_0-mae:174878
[49]	validation_0-mae:172895
[50]	validation_0-mae:171321
[51]	validation_0-mae:170080
[52]	validation_0-mae:168658
[53]	validation_0-mae:167116
[54]	validation_0-mae:165979
[55]	validation_0-mae:163767
[56]	validation_0-mae:162726
[57]	validation_0-mae:160946
[58]	validation_0-mae:159730
[59]	validation_0-mae:158537
[60]	validation_0-mae:157311
[61]	validation_0-mae:156323
[62]	validation_0-mae:155157
[63]	validation_0-mae:154456
[64]	validation_0-mae:153728
[65]	validation_0-mae:152857
[66]	validation_0-mae:152253
[67]	validation_0-mae:151566
[68]	validation_0-mae:149581
[69]	validation_0-mae:148592
[70]	validatio

In [8]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)

df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)

df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

df_test_f, cols_tipodepropiedad_ohe = columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)

df_train_f = columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)

llenar_nulls(df_train_f)
llenar_nulls(df_test_f)

In [15]:
s_train = stack.transform(utils.filtrar_features(df_train_f.drop('precio', axis=1), features))
s_test = stack.transform(utils.filtrar_features(df_test_f, features))

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE



## Prediccion con todos los features + stacking

In [16]:
df_train_s = df_train_f.copy()
df_test_s = df_test_f.copy()

df_train_s['stack01'], df_train_s['stack02'] = zip(*s_train)
df_test_s['stack01'], df_test_s['stack02'] = zip(*s_test)

In [17]:
df_train_s['id'] = df_train['id']
df_test_s['id'] = df_test['id']

In [23]:
params_2nd = {'bagging_fraction': 0.7740520226030885,
 'bagging_freq': int(7.0),
 'feature_fraction': 0.8422472893793045,
 'learning_rate': 0.1508386725397851,
 'max_depth': int(12.0),
 'num_leaves': int(110.0)}

lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(utils.filtrar_features(df_train_s, features + ['stack01', 'stack02']), df_train['precio'].values)

LightGBMWrapper(bagging_fraction=0.7740520226030885, bagging_freq=7,
                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                feature_fraction=0.8422472893793045, importance_type='split',
                learning_rate=0.1508386725397851, max_depth=12,
                min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=110,
                objective=None, random_state=None, reg_alpha=0.0,
                reg_lambda=0.0, silent=True, subsample=1.0,
                subsample_for_bin=200000, subsample_freq=0)

In [25]:
df_test_s['target'] = lgb_m_2nd.predict(utils.filtrar_features(df_test_s, features + ['stack01', 'stack02']))
df_test_s[['id', 'target']].to_csv('respuesta19.csv', index = False)

In [7]:
s_train = stack.transform(x_train)
s_test = stack.transform(x_test)

Train set was detected.
Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE



## Prediccion solo con features de stacking

In [7]:
params_2nd = {'bagging_fraction': 0.8924398062087346,
 'bagging_freq': int(36.0),
 'feature_fraction': 0.16167385124183287,
 'learning_rate': 0.054693418899570134,
 'max_depth': int(4.0),
 'num_leaves': int(93.0)}

lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(s_train, y_train)

LightGBMWrapper(bagging_fraction=0.8924398062087346, bagging_freq=36,
                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                feature_fraction=0.16167385124183287, importance_type='split',
                learning_rate=0.054693418899570134, max_depth=4,
                min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=93,
                objective=None, random_state=None, reg_alpha=0.0,
                reg_lambda=0.0, silent=True, subsample=1.0,
                subsample_for_bin=200000, subsample_freq=0)

In [8]:
keras_mae_train = utils.MAE(y_train, lgb_m_2nd.predict(s_train))
keras_mae_test = utils.MAE(y_test, lgb_m_2nd.predict(s_test))
print(f"MAE Stacking (train): {keras_mae_train:.5f}")
print(f"MAE Stacking (test): {keras_mae_test:.5f}")

MAE Stacking (train): 490724.57530
MAE Stacking (test): 444227.92242


In [12]:
s_test_f = stack.transform(utils.filtrar_features(df_test_f, features))
y_pred_test_f = lgb_m_2nd.predict(s_test_f)
df_test_f['target'] = y_pred_test_f
df_test_f[['id', 'target']].to_csv('respuesta18.csv', index = False)

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE



In [13]:
features = ['stack01', 'stack02', 'stack03']

def eval_lightgbm(args):
    num_leaves, learning_rate, feature_fraction, bagging_fraction, bagging_freq, max_depth = args

    lgb_train = lgb.Dataset(s_train, y_train)
#     lgb_eval = lgb.Dataset(s_test, y_test, reference=lgb_train)
    
    num_leaves = int(num_leaves)
    bagging_freq = int(bagging_freq)
    max_depth = int(max_depth)
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': bagging_freq,
        'max_depth': max_depth,
        'verbose': -1,
    }

    gbm = lgb.train(params,
                    lgb_train,
#                     valid_sets=lgb_eval,
                    num_boost_round=250,
#                     early_stopping_rounds=15,
                    verbose_eval=-1)
    
    y_pred_test = gbm.predict(s_test, num_iteration=gbm.best_iteration)
    return utils.MAE(y_test, y_pred_test)

space = [hp.quniform('num_leaves', 30, 130, 1), hp.uniform('learning_rate', 0.05, 0.9),
        hp.uniform('feature_fraction', 0.10, 0.90), hp.uniform('bagging_fraction', 0.10, 0.90),
        hp.quniform('bagging_freq', 1, 130, 1), hp.quniform('max_depth', 1, 20, 1)]

hps = fmin(eval_lightgbm, space=space, algo=tpe.suggest, max_evals=100, verbose=1)

display(hps)

 12%|█▏        | 12/100 [00:48<05:53,  4.02s/it, best loss: 437714.00137358136]


KeyboardInterrupt: 

In [141]:

# keras_mae_train = utils.MAE(y_test, lgb_m.predict(x_test_s))
# print(f"MAE Keras (train): {keras_mae_train:.5f}")

MAE Keras (train): 524925.45271
